In [1]:
from itertools import product

import numpy as np

from pyinverse.grid import RegularGrid
from pyinverse.axes import RegularAxes3
from pyinverse.radon3 import regular_axes2polytope, grid_uv2half_planes, beam2actor
from pyinverse.volume import volume_cal

from pyviz3d.viz import Renderer

In [2]:
#Nx = Ny = Nz = 9
Nx = 9 * 2
Ny = 10 * 2
Nz = 11 * 2
axes3 = RegularAxes3.linspace((-1, 1, Nx), (-1, 1, Ny), (-1, 1, Nz))

In [3]:
Nu = Nv = 11

mn = (4, 5)

#phi_deg = 15
#theta_deg = 25
phi_deg = 45
theta_deg = 25

grid_uv = RegularGrid.linspace((-1, 1, Nu), (-1, 1, Nv))

In [4]:
A_mn, b_mn = grid_uv2half_planes(theta_deg, phi_deg, grid_uv, mn, degrees=True)

In [5]:
def radon_row(A_mn, b_mn, u_T, v_T, axes3):
    Nz, Ny, Nx = axes3.shape
    
    data = []
    indices = []
    
    for kji in product(range(Nx), range(Ny), range(Nz)):
        ijk = kji[::-1]
        A_ijk, b_ijk = regular_axes2polytope(axes3, ijk)

        A_lass = np.vstack((A_ijk, A_mn))
        b_lass = np.hstack((b_ijk, b_mn))
        
        vol = volume_cal(10, 3, A_lass, b_lass) / (u_T * v_T)
        if not np.allclose(vol, 0):
            data.append(vol)
            indices.append(RegularAxes3.ravel_multi_index(([ijk[2]], [ijk[1]], [ijk[0]]), axes3.shape)[0])
    
    return data, indices

In [6]:
data, indices = radon_row(A_mn, b_mn, grid_uv.axis_x.T, grid_uv.axis_y.T, axes3)

In [7]:
e_min_max = (-1.2*np.sqrt(2), 1.2*np.sqrt(2))
beam_actor = beam2actor(grid_uv, mn, e_min_max, theta_deg, phi_deg, deg=True)

In [8]:
X = np.zeros(axes3.shape)
X.flat[indices] = data

In [9]:
blank_cells = []
ijk_set = set(zip(*RegularAxes3.unravel_indices(indices, axes3.shape)))

for ijk in product(range(Nx), range(Ny), range(Nz)):
    if ijk not in ijk_set:
        blank_cells.append(ijk)

In [10]:
X_actor = axes3.actor(X, blank_cells=blank_cells)

In [11]:
ren = Renderer()
ren.add_actor(beam_actor)
ren.add_actor(X_actor)
ren.axes_on((-1, 1, -1, 1, -1, 1))
ren.reset_camera()
ren.start()